In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

Business Problem

Medicare is a health insurance program run by the U.S. government. The program benefits vulnerable populations such as people aged 65 years and older, people with disabilities, and people with end-stage kidney diseaset [1]. However, fraudulent Medicare claims by providers have been reported, and such fraud contributes to increases in healthcare costs and taxes [2]. In this article, prediction of fraudulent Medicare claims using maching learning will be presented.

Metrics

AUC will be the primary metric to be considered as it measures performance through all possible classification threshold [4]. Overall accuracy and confusion matrix will be also used to measure precision and recall.

Executive Summary A K-Neighbor Classifier model was used to obtain >99% AUC and accuracy of the validation set. The model was able to distinguish fraudulent and nonfraudulent claims with 0% false negative and 0.2% false positive.

Import Related Modules

In [7]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

linkcode
Read CSVs, Clean the Column Names, and Find Shape, NaN, Info

As we will see below, only train dataset has the label (fraudulent or not). Therefore, we will use the train dataset and split the dataset into train and validation sets later.

In [8]:
trnp=pd.read_csv("../input/fraudlent-claim-in-healthcare/Train-1542865627584.csv")
trnb=pd.read_csv("../input/fraudlent-claim-in-healthcare/Train_Beneficiarydata-1542865627584.csv")
trni=pd.read_csv("../input/fraudlent-claim-in-healthcare/Train_Inpatientdata-1542865627584.csv")
trno=pd.read_csv("../input/fraudlent-claim-in-healthcare/Train_Outpatientdata-1542865627584.csv")

tstp=pd.read_csv("../input/fraudlent-claim-in-healthcare/Test-1542969243754.csv")
tstb=pd.read_csv("../input/fraudlent-claim-in-healthcare/Test_Beneficiarydata-1542969243754.csv")
tsti=pd.read_csv("../input/fraudlent-claim-in-healthcare/Test_Inpatientdata-1542969243754.csv")
tsto=pd.read_csv("../input/fraudlent-claim-in-healthcare/Test_Outpatientdata-1542969243754.csv")

In [9]:
# Clean the column names (lower case, remove spaces and special chars)
def col_header_clean(df):
  df.columns=df.columns.str.lower()
  df.columns=df.columns.str.replace(' ','',regex=True)
  df.columns=df.columns.str.replace('[^\w]','_',regex=True)

dfs=[trnp,trnb,trni,trno,tstp,tstb,tsti,tsto]

for i in dfs:
  col_header_clean(i)

In [10]:
# Preview the tables
for i in dfs:
  print(i.head(2))

   provider potentialfraud
0  PRV51001             No
1  PRV51003            Yes
      beneid         dob  dod  gender  race renaldiseaseindicator  state  \
0  BENE11001  1943-01-01  NaN       1     1                     0     39   
1  BENE11002  1936-09-01  NaN       2     1                     0     39   

   county  noofmonths_partacov  noofmonths_partbcov  ...  \
0     230                   12                   12  ...   
1     280                   12                   12  ...   

   chroniccond_depression  chroniccond_diabetes  chroniccond_ischemicheart  \
0                       1                     1                          1   
1                       2                     2                          2   

   chroniccond_osteoporasis  chroniccond_rheumatoidarthritis  \
0                         2                                1   
1                         2                                2   

   chroniccond_stroke  ipannualreimbursementamt  ipannualdeductibleamt  \
0      

provider potentialfraud
0  PRV51001             No
1  PRV51003            Yes
      beneid         dob  dod  gender  race renaldiseaseindicator  state  \
0  BENE11001  1943-01-01  NaN       1     1                     0     39   
1  BENE11002  1936-09-01  NaN       2     1                     0     39   

   county  noofmonths_partacov  noofmonths_partbcov  ...  \
0     230                   12                   12  ...   
1     280                   12                   12  ...   

   chroniccond_depression  chroniccond_diabetes  chroniccond_ischemicheart  \
0                       1                     1                          1   
1                       2                     2                          2   

   chroniccond_osteoporasis  chroniccond_rheumatoidarthritis  \
0                         2                                1   
1                         2                                2   

   chroniccond_stroke  ipannualreimbursementamt  ipannualdeductibleamt  \
0                   1                     36000                   3204   
1                   2                         0                      0   

   opannualreimbursementamt  opannualdeductibleamt  
0                        60                     70  
1                        30                     50  

[2 rows x 25 columns]
      beneid   claimid claimstartdt  claimenddt  provider  \
0  BENE11001  CLM46614   2009-04-12  2009-04-18  PRV55912   
1  BENE11001  CLM66048   2009-08-31  2009-09-02  PRV55907   

   inscclaimamtreimbursed attendingphysician operatingphysician  \
0                   26000          PHY390922                NaN   
1                    5000          PHY318495          PHY318495   

  otherphysician admissiondt  ... clmdiagnosiscode_7  clmdiagnosiscode_8  \
0            NaN  2009-04-12  ...               2724               19889   
1            NaN  2009-08-31  ...                NaN                 NaN   

  clmdiagnosiscode_9 clmdiagnosiscode_10 clmprocedurecode_1  \
0               5849                 NaN                NaN   
1                NaN                 NaN             7092.0   

  clmprocedurecode_2 clmprocedurecode_3 clmprocedurecode_4 clmprocedurecode_5  \
0                NaN                NaN                NaN                NaN   
1                NaN                NaN                NaN                NaN   

  clmprocedurecode_6  
0                NaN  
1                NaN  

[2 rows x 30 columns]
      beneid    claimid claimstartdt  claimenddt  provider  \
0  BENE11002  CLM624349   2009-10-11  2009-10-11  PRV56011   
1  BENE11003  CLM189947   2009-02-12  2009-02-12  PRV57610   

   inscclaimamtreimbursed attendingphysician operatingphysician  \
0                      30          PHY326117                NaN   
1                      80          PHY362868                NaN   

  otherphysician clmdiagnosiscode_1  ... clmdiagnosiscode_9  \
0            NaN              78943  ...                NaN   
1            NaN               6115  ...                NaN   

  clmdiagnosiscode_10 clmprocedurecode_1 clmprocedurecode_2  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   

  clmprocedurecode_3 clmprocedurecode_4 clmprocedurecode_5 clmprocedurecode_6  \
0                NaN                NaN                NaN                NaN   
1                NaN                NaN                NaN                NaN   

  deductibleamtpaid  clmadmitdiagnosiscode  
0                 0                  56409  
1                 0                  79380  

[2 rows x 27 columns]
   provider
0  PRV51002
1  PRV51006
      beneid         dob         dod  gender  race renaldiseaseindicator  \
0  BENE11001  1943-01-01         NaN       1     1                     0   
1  BENE11007  1940-09-01  2009-12-01       1     2                     0   

   state  county  noofmonths_partacov  noofmonths_partbcov  ...  \
0     39     230                   12                   12  ...   
1     45     610                   12                   12  ...   

   chroniccond_depression  chroniccond_diabetes  chroniccond_ischemicheart  \
0                       1                     1                          1   
1                       2                     1                          2   

   chroniccond_osteoporasis  chroniccond_rheumatoidarthritis  \
0                         2                                1   
1                         1                                1   

   chroniccond_stroke  ipannualreimbursementamt  ipannualdeductibleamt  \
0                   1                     36000                   3204   
1                   2                         0                      0   

   opannualreimbursementamt  opannualdeductibleamt  
0                        60                     70  
1                      1490                    160  

[2 rows x 25 columns]
      beneid   claimid claimstartdt  claimenddt  provider  \
0  BENE11014  CLM67387   2009-09-09  2009-09-16  PRV57070   
1  BENE11017  CLM31237   2008-12-25  2009-01-08  PRV54750   

   inscclaimamtreimbursed attendingphysician operatingphysician  \
0                    9000          PHY317786          PHY427017   
1                   14000          PHY314656          PHY426644   

  otherphysician admissiondt  ... clmdiagnosiscode_7  clmdiagnosiscode_8  \
0            NaN  2009-09-09  ...               5990                 570   
1            NaN  2008-12-25  ...               4549               29570   

  clmdiagnosiscode_9 clmdiagnosiscode_10 clmprocedurecode_1  \
0              41071                4280             4443.0   
1              34831                 NaN             5551.0   

  clmprocedurecode_2 clmprocedurecode_3 clmprocedurecode_4 clmprocedurecode_5  \
0             5849.0                NaN                NaN                NaN   
1                NaN                NaN                NaN                NaN   

  clmprocedurecode_6  
0                NaN  
1                NaN  

[2 rows x 30 columns]
      beneid    claimid claimstartdt  claimenddt  provider  \
0  BENE11001  CLM392397   2009-06-02  2009-06-02  PRV55962   
1  BENE11001  CLM430760   2009-06-23  2009-06-23  PRV56112   

   inscclaimamtreimbursed attendingphysician operatingphysician  \
0                      30          PHY347633                NaN   
1                      30          PHY381777                NaN   

  otherphysician clmdiagnosiscode_1  ... clmdiagnosiscode_9  \
0      PHY347633              V5832  ...                NaN   
1      PHY381777               9594  ...                NaN   

  clmdiagnosiscode_10 clmprocedurecode_1 clmprocedurecode_2  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   

  clmprocedurecode_3 clmprocedurecode_4 clmprocedurecode_5 clmprocedurecode_6  \
0                NaN                NaN                NaN                NaN   
1                NaN                NaN                NaN                NaN   

  deductibleamtpaid  clmadmitdiagnosiscode  
0                 0                    NaN  
1                 0                    NaN  

[2 rows x 27 columns]

In [11]:
#Check for null values
for i in dfs:
  print(i.isna().sum())

provider          0
potentialfraud    0
dtype: int64
beneid                                  0
dob                                     0
dod                                137135
gender                                  0
race                                    0
renaldiseaseindicator                   0
state                                   0
county                                  0
noofmonths_partacov                     0
noofmonths_partbcov                     0
chroniccond_alzheimer                   0
chroniccond_heartfailure                0
chroniccond_kidneydisease               0
chroniccond_cancer                      0
chroniccond_obstrpulmonary              0
chroniccond_depression                  0
chroniccond_diabetes                    0
chroniccond_ischemicheart               0
chroniccond_osteoporasis                0
chroniccond_rheumatoidarthritis         0
chroniccond_stroke                      0
ipannualreimbursementamt                0
ipannualdeductibleamt  

provider          0
potentialfraud    0
dtype: int64
beneid                                  0
dob                                     0
dod                                137135
gender                                  0
race                                    0
renaldiseaseindicator                   0
state                                   0
county                                  0
noofmonths_partacov                     0
noofmonths_partbcov                     0
chroniccond_alzheimer                   0
chroniccond_heartfailure                0
chroniccond_kidneydisease               0
chroniccond_cancer                      0
chroniccond_obstrpulmonary              0
chroniccond_depression                  0
chroniccond_diabetes                    0
chroniccond_ischemicheart               0
chroniccond_osteoporasis                0
chroniccond_rheumatoidarthritis         0
chroniccond_stroke                      0
ipannualreimbursementamt                0
ipannualdeductibleamt                   0
opannualreimbursementamt                0
opannualdeductibleamt                   0
dtype: int64
beneid                        0
claimid                       0
claimstartdt                  0
claimenddt                    0
provider                      0
inscclaimamtreimbursed        0
attendingphysician          112
operatingphysician        16644
otherphysician            35784
admissiondt                   0
clmadmitdiagnosiscode         0
deductibleamtpaid           899
dischargedt                   0
diagnosisgroupcode            0
clmdiagnosiscode_1            0
clmdiagnosiscode_2          226
clmdiagnosiscode_3          676
clmdiagnosiscode_4         1534
clmdiagnosiscode_5         2894
clmdiagnosiscode_6         4838
clmdiagnosiscode_7         7258
clmdiagnosiscode_8         9942
clmdiagnosiscode_9        13497
clmdiagnosiscode_10       36547
clmprocedurecode_1        17326
clmprocedurecode_2        35020
clmprocedurecode_3        39509
clmprocedurecode_4        40358
clmprocedurecode_5        40465
clmprocedurecode_6        40474
dtype: int64
beneid                         0
claimid                        0
claimstartdt                   0
claimenddt                     0
provider                       0
inscclaimamtreimbursed         0
attendingphysician          1396
operatingphysician        427120
otherphysician            322691
clmdiagnosiscode_1         10453
clmdiagnosiscode_2        195380
clmdiagnosiscode_3        314480
clmdiagnosiscode_4        392141
clmdiagnosiscode_5        443393
clmdiagnosiscode_6        468981
clmdiagnosiscode_7        484776
clmdiagnosiscode_8        494825
clmdiagnosiscode_9        502899
clmdiagnosiscode_10       516654
clmprocedurecode_1        517575
clmprocedurecode_2        517701
clmprocedurecode_3        517733
clmprocedurecode_4        517735
clmprocedurecode_5        517737
clmprocedurecode_6        517737
deductibleamtpaid              0
clmadmitdiagnosiscode     412312
dtype: int64
provider    0
dtype: int64
beneid                                 0
dob                                    0
dod                                63394
gender                                 0
race                                   0
renaldiseaseindicator                  0
state                                  0
county                                 0
noofmonths_partacov                    0
noofmonths_partbcov                    0
chroniccond_alzheimer                  0
chroniccond_heartfailure               0
chroniccond_kidneydisease              0
chroniccond_cancer                     0
chroniccond_obstrpulmonary             0
chroniccond_depression                 0
chroniccond_diabetes                   0
chroniccond_ischemicheart              0
chroniccond_osteoporasis               0
chroniccond_rheumatoidarthritis        0
chroniccond_stroke                     0
ipannualreimbursementamt               0
ipannualdeductibleamt                  0
opannualreimbursementamt               0
opannualdeductibleamt                  0
dtype: int64
beneid                       0
claimid                      0
claimstartdt                 0
claimenddt                   0
provider                     0
inscclaimamtreimbursed       0
attendingphysician          31
operatingphysician        3962
otherphysician            8538
admissiondt                  0
clmadmitdiagnosiscode        0
deductibleamtpaid          196
dischargedt                  0
diagnosisgroupcode           0
clmdiagnosiscode_1           0
clmdiagnosiscode_2          54
clmdiagnosiscode_3         169
clmdiagnosiscode_4         404
clmdiagnosiscode_5         719
clmdiagnosiscode_6        1197
clmdiagnosiscode_7        1736
clmdiagnosiscode_8        2360
clmdiagnosiscode_9        3238
clmdiagnosiscode_10       8664
clmprocedurecode_1        4118
clmprocedurecode_2        8297
clmprocedurecode_3        9328
clmprocedurecode_4        9522
clmprocedurecode_5        9549
clmprocedurecode_6        9551
dtype: int64
beneid                         0
claimid                        0
claimstartdt                   0
claimenddt                     0
provider                       0
inscclaimamtreimbursed         0
attendingphysician           316
operatingphysician        104237
otherphysician             78222
clmdiagnosiscode_1          2578
clmdiagnosiscode_2         47731
clmdiagnosiscode_3         76575
clmdiagnosiscode_4         95371
clmdiagnosiscode_5        107875
clmdiagnosiscode_6        114035
clmdiagnosiscode_7        117871
clmdiagnosiscode_8        120310
clmdiagnosiscode_9        122278
clmdiagnosiscode_10       125578
clmprocedurecode_1        125807
clmprocedurecode_2        125832
clmprocedurecode_3        125839
clmprocedurecode_4        125841
clmprocedurecode_5        125841
clmprocedurecode_6        125841
deductibleamtpaid              0
clmadmitdiagnosiscode     100036
dtype: int64

In [12]:
#Duplicates of the tables
for i in dfs:
  print(i.duplicated().sum())

0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0

In [13]:
#Shapes of the tables
for i in dfs:
  print(i.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   provider        5410 non-null   object
 1   potentialfraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   beneid                           138556 non-null  object
 1   dob                              138556 non-null  object
 2   dod                              1421 non-null    object
 3   gender                           138556 non-null  int64 
 4   race                             138556 non-null  int64 
 5   renaldiseaseindicator            138556 non-null  object
 6   state                            138556 non-null  int64 
 7 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   provider        5410 non-null   object
 1   potentialfraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   beneid                           138556 non-null  object
 1   dob                              138556 non-null  object
 2   dod                              1421 non-null    object
 3   gender                           138556 non-null  int64 
 4   race                             138556 non-null  int64 
 5   renaldiseaseindicator            138556 non-null  object
 6   state                            138556 non-null  int64 
 7   county                           138556 non-null  int64 
 8   noofmonths_partacov              138556 non-null  int64 
 9   noofmonths_partbcov              138556 non-null  int64 
 10  chroniccond_alzheimer            138556 non-null  int64 
 11  chroniccond_heartfailure         138556 non-null  int64 
 12  chroniccond_kidneydisease        138556 non-null  int64 
 13  chroniccond_cancer               138556 non-null  int64 
 14  chroniccond_obstrpulmonary       138556 non-null  int64 
 15  chroniccond_depression           138556 non-null  int64 
 16  chroniccond_diabetes             138556 non-null  int64 
 17  chroniccond_ischemicheart        138556 non-null  int64 
 18  chroniccond_osteoporasis         138556 non-null  int64 
 19  chroniccond_rheumatoidarthritis  138556 non-null  int64 
 20  chroniccond_stroke               138556 non-null  int64 
 21  ipannualreimbursementamt         138556 non-null  int64 
 22  ipannualdeductibleamt            138556 non-null  int64 
 23  opannualreimbursementamt         138556 non-null  int64 
 24  opannualdeductibleamt            138556 non-null  int64 
dtypes: int64(21), object(4)
memory usage: 26.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   beneid                  40474 non-null  object 
 1   claimid                 40474 non-null  object 
 2   claimstartdt            40474 non-null  object 
 3   claimenddt              40474 non-null  object 
 4   provider                40474 non-null  object 
 5   inscclaimamtreimbursed  40474 non-null  int64  
 6   attendingphysician      40362 non-null  object 
 7   operatingphysician      23830 non-null  object 
 8   otherphysician          4690 non-null   object 
 9   admissiondt             40474 non-null  object 
 10  clmadmitdiagnosiscode   40474 non-null  object 
 11  deductibleamtpaid       39575 non-null  float64
 12  dischargedt             40474 non-null  object 
 13  diagnosisgroupcode      40474 non-null  object 
 14  clmdiagnosiscode_1      40474 non-null  object 
 15  clmdiagnosiscode_2      40248 non-null  object 
 16  clmdiagnosiscode_3      39798 non-null  object 
 17  clmdiagnosiscode_4      38940 non-null  object 
 18  clmdiagnosiscode_5      37580 non-null  object 
 19  clmdiagnosiscode_6      35636 non-null  object 
 20  clmdiagnosiscode_7      33216 non-null  object 
 21  clmdiagnosiscode_8      30532 non-null  object 
 22  clmdiagnosiscode_9      26977 non-null  object 
 23  clmdiagnosiscode_10     3927 non-null   object 
 24  clmprocedurecode_1      23148 non-null  float64
 25  clmprocedurecode_2      5454 non-null   float64
 26  clmprocedurecode_3      965 non-null    float64
 27  clmprocedurecode_4      116 non-null    float64
 28  clmprocedurecode_5      9 non-null      float64
 29  clmprocedurecode_6      0 non-null      float64
dtypes: float64(7), int64(1), object(22)
memory usage: 9.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   beneid                  517737 non-null  object 
 1   claimid                 517737 non-null  object 
 2   claimstartdt            517737 non-null  object 
 3   claimenddt              517737 non-null  object 
 4   provider                517737 non-null  object 
 5   inscclaimamtreimbursed  517737 non-null  int64  
 6   attendingphysician      516341 non-null  object 
 7   operatingphysician      90617 non-null   object 
 8   otherphysician          195046 non-null  object 
 9   clmdiagnosiscode_1      507284 non-null  object 
 10  clmdiagnosiscode_2      322357 non-null  object 
 11  clmdiagnosiscode_3      203257 non-null  object 
 12  clmdiagnosiscode_4      125596 non-null  object 
 13  clmdiagnosiscode_5      74344 non-null   object 
 14  clmdiagnosiscode_6      48756 non-null   object 
 15  clmdiagnosiscode_7      32961 non-null   object 
 16  clmdiagnosiscode_8      22912 non-null   object 
 17  clmdiagnosiscode_9      14838 non-null   object 
 18  clmdiagnosiscode_10     1083 non-null    object 
 19  clmprocedurecode_1      162 non-null     float64
 20  clmprocedurecode_2      36 non-null      float64
 21  clmprocedurecode_3      4 non-null       float64
 22  clmprocedurecode_4      2 non-null       float64
 23  clmprocedurecode_5      0 non-null       float64
 24  clmprocedurecode_6      0 non-null       float64
 25  deductibleamtpaid       517737 non-null  int64  
 26  clmadmitdiagnosiscode   105425 non-null  object 
dtypes: float64(6), int64(2), object(19)
memory usage: 106.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353 entries, 0 to 1352
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   provider  1353 non-null   object
dtypes: object(1)
memory usage: 10.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63968 entries, 0 to 63967
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   beneid                           63968 non-null  object
 1   dob                              63968 non-null  object
 2   dod                              574 non-null    object
 3   gender                           63968 non-null  int64 
 4   race                             63968 non-null  int64 
 5   renaldiseaseindicator            63968 non-null  object
 6   state                            63968 non-null  int64 
 7   county                           63968 non-null  int64 
 8   noofmonths_partacov              63968 non-null  int64 
 9   noofmonths_partbcov              63968 non-null  int64 
 10  chroniccond_alzheimer            63968 non-null  int64 
 11  chroniccond_heartfailure         63968 non-null  int64 
 12  chroniccond_kidneydisease        63968 non-null  int64 
 13  chroniccond_cancer               63968 non-null  int64 
 14  chroniccond_obstrpulmonary       63968 non-null  int64 
 15  chroniccond_depression           63968 non-null  int64 
 16  chroniccond_diabetes             63968 non-null  int64 
 17  chroniccond_ischemicheart        63968 non-null  int64 
 18  chroniccond_osteoporasis         63968 non-null  int64 
 19  chroniccond_rheumatoidarthritis  63968 non-null  int64 
 20  chroniccond_stroke               63968 non-null  int64 
 21  ipannualreimbursementamt         63968 non-null  int64 
 22  ipannualdeductibleamt            63968 non-null  int64 
 23  opannualreimbursementamt         63968 non-null  int64 
 24  opannualdeductibleamt            63968 non-null  int64 
dtypes: int64(21), object(4)
memory usage: 12.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   beneid                  9551 non-null   object 
 1   claimid                 9551 non-null   object 
 2   claimstartdt            9551 non-null   object 
 3   claimenddt              9551 non-null   object 
 4   provider                9551 non-null   object 
 5   inscclaimamtreimbursed  9551 non-null   int64  
 6   attendingphysician      9520 non-null   object 
 7   operatingphysician      5589 non-null   object 
 8   otherphysician          1013 non-null   object 
 9   admissiondt             9551 non-null   object 
 10  clmadmitdiagnosiscode   9551 non-null   object 
 11  deductibleamtpaid       9355 non-null   float64
 12  dischargedt             9551 non-null   object 
 13  diagnosisgroupcode      9551 non-null   object 
 14  clmdiagnosiscode_1      9551 non-null   object 
 15  clmdiagnosiscode_2      9497 non-null   object 
 16  clmdiagnosiscode_3      9382 non-null   object 
 17  clmdiagnosiscode_4      9147 non-null   object 
 18  clmdiagnosiscode_5      8832 non-null   object 
 19  clmdiagnosiscode_6      8354 non-null   object 
 20  clmdiagnosiscode_7      7815 non-null   object 
 21  clmdiagnosiscode_8      7191 non-null   object 
 22  clmdiagnosiscode_9      6313 non-null   object 
 23  clmdiagnosiscode_10     887 non-null    object 
 24  clmprocedurecode_1      5433 non-null   float64
 25  clmprocedurecode_2      1254 non-null   float64
 26  clmprocedurecode_3      223 non-null    float64
 27  clmprocedurecode_4      29 non-null     float64
 28  clmprocedurecode_5      2 non-null      float64
 29  clmprocedurecode_6      0 non-null      float64
dtypes: float64(7), int64(1), object(22)
memory usage: 2.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125841 entries, 0 to 125840
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   beneid                  125841 non-null  object 
 1   claimid                 125841 non-null  object 
 2   claimstartdt            125841 non-null  object 
 3   claimenddt              125841 non-null  object 
 4   provider                125841 non-null  object 
 5   inscclaimamtreimbursed  125841 non-null  int64  
 6   attendingphysician      125525 non-null  object 
 7   operatingphysician      21604 non-null   object 
 8   otherphysician          47619 non-null   object 
 9   clmdiagnosiscode_1      123263 non-null  object 
 10  clmdiagnosiscode_2      78110 non-null   object 
 11  clmdiagnosiscode_3      49266 non-null   object 
 12  clmdiagnosiscode_4      30470 non-null   object 
 13  clmdiagnosiscode_5      17966 non-null   object 
 14  clmdiagnosiscode_6      11806 non-null   object 
 15  clmdiagnosiscode_7      7970 non-null    object 
 16  clmdiagnosiscode_8      5531 non-null    object 
 17  clmdiagnosiscode_9      3563 non-null    object 
 18  clmdiagnosiscode_10     263 non-null     object 
 19  clmprocedurecode_1      34 non-null      float64
 20  clmprocedurecode_2      9 non-null       float64
 21  clmprocedurecode_3      2 non-null       float64
 22  clmprocedurecode_4      0 non-null       float64
 23  clmprocedurecode_5      0 non-null       float64
 24  clmprocedurecode_6      0 non-null       float64
 25  deductibleamtpaid       125841 non-null  int64  
 26  clmadmitdiagnosiscode   25805 non-null   object 
dtypes: float64(6), int64(2), object(19)
memory usage: 25.9+ MB
None

Combine Datasets

To make the training set ML processible, we will merge the datasets into one. The inpatient and outpatient sets have similar features, so they can be merged. An inptient indicator column will be created to distinguish inpatient and outpatient claims.

In [14]:
#Merge to make the outpatient dataset

trnoutpt=pd.merge(trno,trnp,on='provider',how='left')
trnoutpt=pd.merge(trnoutpt,trnb,on='beneid')
print(trnoutpt.isna().sum())
print(trnoutpt.shape)

beneid                                  0
claimid                                 0
claimstartdt                            0
claimenddt                              0
provider                                0
inscclaimamtreimbursed                  0
attendingphysician                   1396
operatingphysician                 427120
otherphysician                     322691
clmdiagnosiscode_1                  10453
clmdiagnosiscode_2                 195380
clmdiagnosiscode_3                 314480
clmdiagnosiscode_4                 392141
clmdiagnosiscode_5                 443393
clmdiagnosiscode_6                 468981
clmdiagnosiscode_7                 484776
clmdiagnosiscode_8                 494825
clmdiagnosiscode_9                 502899
clmdiagnosiscode_10                516654
clmprocedurecode_1                 517575
clmprocedurecode_2                 517701
clmprocedurecode_3                 517733
clmprocedurecode_4                 517735
clmprocedurecode_5                

beneid                                  0
claimid                                 0
claimstartdt                            0
claimenddt                              0
provider                                0
inscclaimamtreimbursed                  0
attendingphysician                   1396
operatingphysician                 427120
otherphysician                     322691
clmdiagnosiscode_1                  10453
clmdiagnosiscode_2                 195380
clmdiagnosiscode_3                 314480
clmdiagnosiscode_4                 392141
clmdiagnosiscode_5                 443393
clmdiagnosiscode_6                 468981
clmdiagnosiscode_7                 484776
clmdiagnosiscode_8                 494825
clmdiagnosiscode_9                 502899
clmdiagnosiscode_10                516654
clmprocedurecode_1                 517575
clmprocedurecode_2                 517701
clmprocedurecode_3                 517733
clmprocedurecode_4                 517735
clmprocedurecode_5                 517737
clmprocedurecode_6                 517737
deductibleamtpaid                       0
clmadmitdiagnosiscode              412312
potentialfraud                          0
dob                                     0
dod                                513924
gender                                  0
race                                    0
renaldiseaseindicator                   0
state                                   0
county                                  0
noofmonths_partacov                     0
noofmonths_partbcov                     0
chroniccond_alzheimer                   0
chroniccond_heartfailure                0
chroniccond_kidneydisease               0
chroniccond_cancer                      0
chroniccond_obstrpulmonary              0
chroniccond_depression                  0
chroniccond_diabetes                    0
chroniccond_ischemicheart               0
chroniccond_osteoporasis                0
chroniccond_rheumatoidarthritis         0
chroniccond_stroke                      0
ipannualreimbursementamt                0
ipannualdeductibleamt                   0
opannualreimbursementamt                0
opannualdeductibleamt                   0
dtype: int64
(517737, 52)
In [10]:


In [15]:
#Label 'in/outpatient' for the dataset
trnoutpt['inpt']=0
print(trnoutpt.columns)

Index(['beneid', 'claimid', 'claimstartdt', 'claimenddt', 'provider',
       'inscclaimamtreimbursed', 'attendingphysician', 'operatingphysician',
       'otherphysician', 'clmdiagnosiscode_1', 'clmdiagnosiscode_2',
       'clmdiagnosiscode_3', 'clmdiagnosiscode_4', 'clmdiagnosiscode_5',
       'clmdiagnosiscode_6', 'clmdiagnosiscode_7', 'clmdiagnosiscode_8',
       'clmdiagnosiscode_9', 'clmdiagnosiscode_10', 'clmprocedurecode_1',
       'clmprocedurecode_2', 'clmprocedurecode_3', 'clmprocedurecode_4',
       'clmprocedurecode_5', 'clmprocedurecode_6', 'deductibleamtpaid',
       'clmadmitdiagnosiscode', 'potentialfraud', 'dob', 'dod', 'gender',
       'race', 'renaldiseaseindicator', 'state', 'county',
       'noofmonths_partacov', 'noofmonths_partbcov', 'chroniccond_alzheimer',
       'chroniccond_heartfailure', 'chroniccond_kidneydisease',
       'chroniccond_cancer', 'chroniccond_obstrpulmonary',
       'chroniccond_depression', 'chroniccond_diabetes',
       'chroniccond_ischemic

Index(['beneid', 'claimid', 'claimstartdt', 'claimenddt', 'provider',
       'inscclaimamtreimbursed', 'attendingphysician', 'operatingphysician',
       'otherphysician', 'clmdiagnosiscode_1', 'clmdiagnosiscode_2',
       'clmdiagnosiscode_3', 'clmdiagnosiscode_4', 'clmdiagnosiscode_5',
       'clmdiagnosiscode_6', 'clmdiagnosiscode_7', 'clmdiagnosiscode_8',
       'clmdiagnosiscode_9', 'clmdiagnosiscode_10', 'clmprocedurecode_1',
       'clmprocedurecode_2', 'clmprocedurecode_3', 'clmprocedurecode_4',
       'clmprocedurecode_5', 'clmprocedurecode_6', 'deductibleamtpaid',
       'clmadmitdiagnosiscode', 'potentialfraud', 'dob', 'dod', 'gender',
       'race', 'renaldiseaseindicator', 'state', 'county',
       'noofmonths_partacov', 'noofmonths_partbcov', 'chroniccond_alzheimer',
       'chroniccond_heartfailure', 'chroniccond_kidneydisease',
       'chroniccond_cancer', 'chroniccond_obstrpulmonary',
       'chroniccond_depression', 'chroniccond_diabetes',
       'chroniccond_ischemicheart', 'chroniccond_osteoporasis',
       'chroniccond_rheumatoidarthritis', 'chroniccond_stroke',
       'ipannualreimbursementamt', 'ipannualdeductibleamt',
       'opannualreimbursementamt', 'opannualdeductibleamt', 'inpt'],
      dtype='object')

In [16]:
#Merge to make inpt dataset

trninpt=pd.merge(trni,trnp,on='provider',how='left')
trninpt=pd.merge(trninpt,trnb,on='beneid')
print(trninpt.isna().sum())
print(trninpt.shape)

beneid                                 0
claimid                                0
claimstartdt                           0
claimenddt                             0
provider                               0
inscclaimamtreimbursed                 0
attendingphysician                   112
operatingphysician                 16644
otherphysician                     35784
admissiondt                            0
clmadmitdiagnosiscode                  0
deductibleamtpaid                    899
dischargedt                            0
diagnosisgroupcode                     0
clmdiagnosiscode_1                     0
clmdiagnosiscode_2                   226
clmdiagnosiscode_3                   676
clmdiagnosiscode_4                  1534
clmdiagnosiscode_5                  2894
clmdiagnosiscode_6                  4838
clmdiagnosiscode_7                  7258
clmdiagnosiscode_8                  9942
clmdiagnosiscode_9                 13497
clmdiagnosiscode_10                36547
clmprocedurecode

beneid                                 0
claimid                                0
claimstartdt                           0
claimenddt                             0
provider                               0
inscclaimamtreimbursed                 0
attendingphysician                   112
operatingphysician                 16644
otherphysician                     35784
admissiondt                            0
clmadmitdiagnosiscode                  0
deductibleamtpaid                    899
dischargedt                            0
diagnosisgroupcode                     0
clmdiagnosiscode_1                     0
clmdiagnosiscode_2                   226
clmdiagnosiscode_3                   676
clmdiagnosiscode_4                  1534
clmdiagnosiscode_5                  2894
clmdiagnosiscode_6                  4838
clmdiagnosiscode_7                  7258
clmdiagnosiscode_8                  9942
clmdiagnosiscode_9                 13497
clmdiagnosiscode_10                36547
clmprocedurecode_1                 17326
clmprocedurecode_2                 35020
clmprocedurecode_3                 39509
clmprocedurecode_4                 40358
clmprocedurecode_5                 40465
clmprocedurecode_6                 40474
potentialfraud                         0
dob                                    0
dod                                40156
gender                                 0
race                                   0
renaldiseaseindicator                  0
state                                  0
county                                 0
noofmonths_partacov                    0
noofmonths_partbcov                    0
chroniccond_alzheimer                  0
chroniccond_heartfailure               0
chroniccond_kidneydisease              0
chroniccond_cancer                     0
chroniccond_obstrpulmonary             0
chroniccond_depression                 0
chroniccond_diabetes                   0
chroniccond_ischemicheart              0
chroniccond_osteoporasis               0
chroniccond_rheumatoidarthritis        0
chroniccond_stroke                     0
ipannualreimbursementamt               0
ipannualdeductibleamt                  0
opannualreimbursementamt               0
opannualdeductibleamt                  0
dtype: int64
(40474, 55)

In [17]:
#label 'inpatient' for the dataset
trninpt['inpt']=1
print(trnoutpt.columns)

Index(['beneid', 'claimid', 'claimstartdt', 'claimenddt', 'provider',
       'inscclaimamtreimbursed', 'attendingphysician', 'operatingphysician',
       'otherphysician', 'clmdiagnosiscode_1', 'clmdiagnosiscode_2',
       'clmdiagnosiscode_3', 'clmdiagnosiscode_4', 'clmdiagnosiscode_5',
       'clmdiagnosiscode_6', 'clmdiagnosiscode_7', 'clmdiagnosiscode_8',
       'clmdiagnosiscode_9', 'clmdiagnosiscode_10', 'clmprocedurecode_1',
       'clmprocedurecode_2', 'clmprocedurecode_3', 'clmprocedurecode_4',
       'clmprocedurecode_5', 'clmprocedurecode_6', 'deductibleamtpaid',
       'clmadmitdiagnosiscode', 'potentialfraud', 'dob', 'dod', 'gender',
       'race', 'renaldiseaseindicator', 'state', 'county',
       'noofmonths_partacov', 'noofmonths_partbcov', 'chroniccond_alzheimer',
       'chroniccond_heartfailure', 'chroniccond_kidneydisease',
       'chroniccond_cancer', 'chroniccond_obstrpulmonary',
       'chroniccond_depression', 'chroniccond_diabetes',
       'chroniccond_ischemic

Index(['beneid', 'claimid', 'claimstartdt', 'claimenddt', 'provider',
       'inscclaimamtreimbursed', 'attendingphysician', 'operatingphysician',
       'otherphysician', 'clmdiagnosiscode_1', 'clmdiagnosiscode_2',
       'clmdiagnosiscode_3', 'clmdiagnosiscode_4', 'clmdiagnosiscode_5',
       'clmdiagnosiscode_6', 'clmdiagnosiscode_7', 'clmdiagnosiscode_8',
       'clmdiagnosiscode_9', 'clmdiagnosiscode_10', 'clmprocedurecode_1',
       'clmprocedurecode_2', 'clmprocedurecode_3', 'clmprocedurecode_4',
       'clmprocedurecode_5', 'clmprocedurecode_6', 'deductibleamtpaid',
       'clmadmitdiagnosiscode', 'potentialfraud', 'dob', 'dod', 'gender',
       'race', 'renaldiseaseindicator', 'state', 'county',
       'noofmonths_partacov', 'noofmonths_partbcov', 'chroniccond_alzheimer',
       'chroniccond_heartfailure', 'chroniccond_kidneydisease',
       'chroniccond_cancer', 'chroniccond_obstrpulmonary',
       'chroniccond_depression', 'chroniccond_diabetes',
       'chroniccond_ischemicheart', 'chroniccond_osteoporasis',
       'chroniccond_rheumatoidarthritis', 'chroniccond_stroke',
       'ipannualreimbursementamt', 'ipannualdeductibleamt',
       'opannualreimbursementamt', 'opannualdeductibleamt', 'inpt'],
      dtype='object')

Cleaning of Data & Age Calculation

In this section, values of binary categorical variables ('Y'/'N' or '1'/'2') will be converted to 1 and 0. Also, values with date and time data will be converted to the correct form. Based on the dates, information such as patient's age at the time of claim, duration of the claim, and length of hostpital stay will be calculated.

In [18]:
Change 2 to 0 in chronic disease categories & gender
binary=['chroniccond_alzheimer','chroniccond_heartfailure','chroniccond_kidneydisease',\
         'chroniccond_cancer','chroniccond_obstrpulmonary','chroniccond_depression',\
         'chroniccond_diabetes','chroniccond_ischemicheart','chroniccond_osteoporasis',\
         'chroniccond_rheumatoidarthritis','chroniccond_stroke','gender']
for i in binary:
  trnset[i].replace((2),0,inplace=True)

SyntaxError: invalid syntax (4163857802.py, line 1)

In [ ]:
#Calculate age on the first day of claim
trnset['ageatclaim']=np.floor(((trnset['claimstartdt'] - trnset['dob']).dt.days)/365.25)

#Calculate duration of claim
trnset['duration'] = ((trnset['claimenddt'] - trnset['claimstartdt']).dt.days)+1

#Calculate LOS(length of stay) of admitted patients
trnset['los']=((trnset['dischargedt'] - trnset['admissiondt']).dt.days)+1

Conclusion

The average quantifiable values per prescriber, such as amount of money deducted/reimbursed, duration of the claim, length of the stay were calculated to help find anomalies in prescribers. Because of the extensive number of predictive variables, Recursive Feature Elimination (RFE) was used to select 10 most important features. The dataset was also SMOTEd to balance the ration between fraudulent vs. nonfraudulent. K-Neighbors was used to establish the model. The model was able to achieve AUC and accuracy of 99% using the dataset.